In [ ]:
# Energy from the ME model

In [1]:
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys


In [ ]:
# load de ME model
with open('../files/models/iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

In [ ]:
from energy_analisis import get_energy_consumption_production

In [ ]:
# Get the consumption and production of energy per reaction
reactions_energy_consumption,reactions_energy_production = get_energy_consumption_production(me)

In [ ]:
energy_consumption = pd.DataFrame.from_dict(reactions_energy_consumption).set_index('Reactions')
energy_consumption.head()

In [ ]:
energy_consumption.to_pickle('energy_consumption.pickle')

In [8]:
with open('../files/energy/energy_consumption.pickle', 'rb') as f:
    energy_consumption = pickle.load(f)

In [9]:
data = energy_consumption
data.loc[:,'Used'] = abs(data.Used)
elegidos=data
elegidos.head()

,Type,Total,Used,ATP,GTP
Reactions,,,,,
transcription_TU0_8529_from_RpoD_mono,TranscriptionReaction,-347.0,3.937972e-09,-347.0,0
transcription_TU0_8464_from_RpoD_mono,TranscriptionReaction,-88.0,2.024358e-04,-88.0,0
transcription_TU0_13006_from_RpoD_mono,TranscriptionReaction,-172.0,5.577844e-12,-172.0,0
transcription_TU0_5003_from_RPOH_MONOMER,TranscriptionReaction,-434.0,1.183178e-07,-434.0,0
transcription_TU0_3423_from_RPOE_MONOMER,TranscriptionReaction,-109.0,3.756146e-06,-109.0,0


In [ ]:
elegidos = elegidos[elegidos.Type=='MetabolicReaction']
genes_met={}
faltante_metabolico=0
elegidos_maximos={}
genes_tot=[]
reactant_tipos=[]
Complejos = []
complex_formation = []
for reaction in elegidos.index:
    elegidos_maximos[reaction]=[]
    a =me.reactions.get_by_id(reaction)
    for reactant in a.reactants:
        reactant_tipos.append(type(reactant).__name__)
        if((type(reactant).__name__)) =='Complex':
            Complejos.append(reactant)
            for reaccion_complejo in reactant.reactions:
                if (type(reaccion_complejo).__name__) == 'ComplexFormation':
                    complex_formation.append(reaccion_complejo)
#                     print(reaction,reactant, reaccion_complejo.reactants,'\n')

                    n_genes=0
                    for reactant_complejo in reaccion_complejo.reactants:
                         # Ver si solo hay un gen
#                         print(reactant_complejo,'miau')
#                         print(type(reactant_complejo).__name__,'miau2')
                        if (type(reactant_complejo).__name__) == 'TranslatedGene':
#                             print(reaction,reactant_complejo,'\n')
                            elegidos_maximos[reaction].append(reactant_complejo.id.split('_')[1])
                            genes_tot.append(reactant_complejo.id.split('_')[1])
                            n_genes+=1
                    # Calculo real tomando en cuenta cuanto aporta cada gen
#                     print(n_genes)
                    if n_genes > 1:
                        total = 0
                        valores = {}
                        for reactant_complejo in reaccion_complejo.reactants:
                            if (type(reactant_complejo).__name__) == 'TranslatedGene':
                                                    valores[reactant_complejo.id] = abs(reaccion_complejo.get_coefficient(reactant_complejo))
                        total = np.sum(list(valores.values()))
    
                        for key,value in valores.items():
                            bnumber_m= key.split('_')[1]
                            genes_met[bnumber_m] = genes_met.get(bnumber_m, 0) + value/total*a.x
                    elif n_genes == 1:
                        genes_met[bnumber_m] = genes_met.get(bnumber_m, 0) + a.x
                    else:
                        faltante_metabolico+=(a.x)



In [ ]:
transcription = data.Used[data.Type=="TranscriptionReaction"]
genes_transcription={}
for ind in transcription.index:
    
    rxn = me.reactions.get_by_id(ind)
    num_gen= 0
    lista_genes = []
    for gene in rxn.products:
        
        if type(gene).__name__ == 'TranscribedGene':
            lista_genes.append(gene.id)
    if len(lista_genes) > 1:
                    total = 0
                    valores = {}
                    for gene in lista_genes:
                           valores[gene] = abs(rxn.get_coefficient(gene))
                            
                    total = np.sum(list(valores.values()))
    
                    for key,value in valores.items():
                        bnumber_m= key.split('_')[1]
                        genes_transcription[bnumber_m] = genes_transcription.get(bnumber_m, 0) + value/total*rxn.x
    elif len(lista_genes) == 1:
                        bnumber_m= lista_genes[0].split('_')[1]
                        genes_transcription[bnumber_m] = genes_transcription.get(bnumber_m, 0) + rxn.x
    else:
        print(rxn.x)

In [ ]:
elegidos_trad = data.loc[data.Type=='TranslationReaction']
# transl_w3110 = ['translation_'+ gen for gen in genes_W3110]
# energia_translation = elegidos_trad.loc[elegidos_trad.index.isin(transl_w3110)]
elegidos_trad = elegidos_trad.Used


for trans_bnumber in elegidos_trad.index:
    bnumber=trans_bnumber.split('_')[1]
    genes_met[bnumber] = genes_met.get(bnumber, 0) + elegidos_trad.loc[trans_bnumber]

In [10]:
Energia_total_ME = pd.DataFrame(genes_met.items(), columns=['Bnumber', 'Value'])
Energia_total_ME = Energia_total_ME.sort_values(by='Value',ascending=False).reset_index()
Energia_total_ME = Energia_total_ME.loc[:,['Bnumber','Value']]

NameError: name 'genes_met' is not defined

In [ ]:
Energia_W3110 = Energia_total_ME
Energia_W3110 = Energia_W3110.sort_values(by='Value',ascending=False).reset_index()
Energia_W3110 = Energia_W3110.loc[:,['Bnumber','Value']]

In [ ]:
Energia_W3110.to_pickle('Energia_por_gen.pickle')